In [1]:
%cd ../..
%pwd

/root/CoTFaithChecker


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/root/CoTFaithChecker'

In [2]:
%ls h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/*

h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_0.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_128.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_160.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_192.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_224.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_256.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_288.pt
h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/none/500_captures/first_token_acts_batch_32.pt
h_hidden_sp

In [3]:
import os, pickle, datetime, re
from pathlib import Path

import torch, numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler

ROOT = Path("h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/")
FOLDER_NAME = "500_captures"
DIR_NONE        = ROOT / "none" / FOLDER_NAME
DIR_unverb  = ROOT / "sycophancy" / FOLDER_NAME

def list_pt(directory):
    return sorted(str(p) for p in directory.glob("*.pt"))

files_none  = list_pt(DIR_NONE)
files_sync  = list_pt(DIR_unverb)
assert files_none and files_sync, "No .pt files found ! check paths"


In [4]:
pointer_re = re.compile(br"^version https://git-lfs.github.com/spec/")
def safe_torch_load(fname):
    with open(fname, "rb") as f:
        if pointer_re.match(f.read(80)):
            print(f"{Path(fname).name}: Git-LFS pointer, skipping")
            return None
    return torch.load(fname, map_location="cpu")


In [5]:
def dict_to_list(d):
    return [d[f"layer_{i}"] for i in range(len(d))]
def normalise_batch(obj):
    if isinstance(obj, dict):
        return dict_to_list(obj)
    elif isinstance(obj, (list, tuple)):
        return list(obj)
    raise TypeError(f"Unexpected batch type: {type(obj)}")

sample = None
for fp in files_none + files_sync:
    raw = safe_torch_load(fp)
    if raw is not None:
        sample = normalise_batch(raw)
        break

assert sample is not None, "No real .pt blobs present – run `git lfs pull`"
N_LAYERS     = len(sample)
HIDDEN_SIZE  = sample[0].shape[-1]
print(f"Detected {N_LAYERS} layers, hidden size = {HIDDEN_SIZE}")



Detected 33 layers, hidden size = 4096


/tmp/ipykernel_29325/1195087584.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(fname, map_location="cpu")


In [6]:
layer_blobs = {L: [] for L in range(N_LAYERS)}
labels      = []

def add_files(file_list, lab):
    for fp in tqdm(file_list, desc=f"label={lab}"):
        raw = safe_torch_load(fp)
        if raw is None:
            continue
        batch = normalise_batch(raw)
        B = batch[0].shape[0]
        for L, h in enumerate(batch):
            layer_blobs[L].append(h.float().numpy())
        labels.extend([lab]*B)

add_files(files_none,  0)   # 0 = none
add_files(files_sync,  1)   # 1 = unverb

assert labels, "No usable data loaded."
labels = np.asarray(labels, dtype=np.int8)
print("Total samples:", len(labels))

layer_X = {L: np.concatenate(layer_blobs[L], axis=0) for L in layer_blobs}
del layer_blobs

scalers = {}
for L in range(N_LAYERS):
    scaler = StandardScaler()
    layer_X[L] = scaler.fit_transform(layer_X[L])
    scalers[L] = scaler

label=0:   0%|          | 0/16 [00:00<?, ?it/s]/tmp/ipykernel_29325/1195087584.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(fname, map_location="cpu

Total samples: 1000


In [7]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
layer_scores = []

print("\nCross-validation accuracy:")
for L in range(N_LAYERS):
    clf = LogisticRegression(penalty="l2", C=1.0, max_iter=1000, n_jobs=-1)
    acc = cross_val_score(clf, layer_X[L], labels, cv=cv, scoring="accuracy").mean()
    print(f"layer {L:2d}:  {acc:.3f}")
    layer_scores.append(acc)

best_layer = int(np.argmax(layer_scores))
print(f"\First highest layer = {best_layer}  (acc = {layer_scores[best_layer]:.3f})")

# final fit on the best layer, save probe + Δµ
X_best  = layer_X[best_layer]
clf_best = LogisticRegression(penalty="l2", C=1.0, max_iter=1000, n_jobs=-1).fit(X_best, labels)

mu_none = X_best[labels == 0].mean(axis=0)
mu_sync = X_best[labels == 1].mean(axis=0)
delta_mu = mu_sync - mu_none

OUT_PROBE = ROOT / f"unverb_probe_layer{best_layer}.pkl"
with open(OUT_PROBE, "wb") as f:
    pickle.dump(
        dict(
            layer       = best_layer,
            weights     = clf_best.coef_[0].astype(np.float32),
            intercept   = float(clf_best.intercept_[0]),
            delta_mu    = delta_mu.astype(np.float32),
            hidden_size = HIDDEN_SIZE,
            created     = datetime.datetime.now().isoformat(timespec="seconds"),
            acc_cv      = float(layer_scores[best_layer]),
            note        = "0 = none, 1 = unverb; StandardScaler applied.",
        ),
        f,
    )
print(f"\nProbe saved to {OUT_PROBE.relative_to(Path('.'))}")



Cross-validation accuracy:


layer  0:  0.678
layer  1:  0.973
layer  2:  0.984
layer  3:  0.992
layer  4:  0.996
layer  5:  0.996
layer  6:  0.997
layer  7:  0.999
layer  8:  1.000
layer  9:  1.000
layer 10:  1.000
layer 11:  1.000
layer 12:  1.000
layer 13:  1.000
layer 14:  1.000
layer 15:  1.000
layer 16:  1.000
layer 17:  1.000
layer 18:  1.000
layer 19:  1.000
layer 20:  1.000
layer 21:  1.000
layer 22:  1.000
layer 23:  1.000
layer 24:  1.000
layer 25:  1.000
layer 26:  1.000
layer 27:  1.000
layer 28:  1.000
layer 29:  1.000
layer 30:  1.000
layer 31:  1.000
layer 32:  1.000
\First highest layer = 8  (acc = 1.000)

Probe saved to h_hidden_space/outputs/f1_hint_xyyx/hints/mmlu/DeepSeek-R1-Distill-Llama-8B/unverb_probe_layer8.pkl


In [8]:
from i_probe_steer.main.run_and_steer import run_steered_generation

results = run_steered_generation(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    probe_path=OUT_PROBE,
    dataset_name="mmlu",
    hint_type="none",
    n_questions=1,
    alpha=5.0,               # larger = stronger steer
    batch_size=8,
    max_new_tokens=256,
    output_jsonl="steered_50_mmlu_none.jsonl"
)

results[:1]


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-30 17:49:37,696 - INFO - CUDA is available. Using GPU.
2025-04-30 17:49:37,697 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Llama-8B onto cuda
Fetching 2 files: 100%|██████████| 2/2 [00:44<00:00, 22.48s/it]
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:817: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.83it/s]
2025-04-30 17:50:32,946 - INFO - Model and tokenizer loaded successfully.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py

TypeError: can only concatenate tuple (not "Tensor") to tuple